In [1]:
# !pip install imbalanced-learn

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
import seaborn as sns
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

In [18]:
df=pd.read_csv('C:\Data\DJ\AccioBeer\data\EEG.machinelearing_data_BRMH.csv')
df.head()

,no.,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
0,1,M,57.0,2012.8.30,NaN,NaN,Addictive disorder,Alcohol use disorder,35.998557,21.717375,...,55.989192,16.739679,23.452271,45.678820,30.167520,16.918761,48.850427,9.422630,34.507082,28.613029
1,2,M,37.0,2012.9.6,6.0,120.0,Addictive disorder,Alcohol use disorder,13.425118,11.002916,...,45.595619,17.510824,26.777368,28.201062,57.108861,32.375401,60.351749,13.900981,57.831848,43.463261
2,3,M,32.0,2012.9.10,16.0,113.0,Addictive disorder,Alcohol use disorder,29.941780,27.544684,...,99.475453,70.654171,39.131547,69.920996,71.063644,38.534505,69.908764,27.180532,64.803155,31.485799
3,4,M,35.0,2012.10.8,18.0,126.0,Addictive disorder,Alcohol use disorder,21.496226,21.846832,...,59.986561,63.822201,36.478254,47.117006,84.658376,24.724096,50.299349,35.319695,79.822944,41.141873
4,5,M,36.0,2012.10.18,16.0,112.0,Addictive disorder,Alcohol use disorder,37.775667,33.607679,...,61.462720,59.166097,51.465531,58.635415,80.685608,62.138436,75.888749,61.003944,87.455509,70.531662


In [19]:
df = df.drop(columns=['no.', 'eeg.date'], axis=1)

In [20]:
# df = df.fillna(df.mean())
# df['sex'].fillna(value='M', inplace=True)
df['sex'].fillna('M')

0      M
1      M
2      M
3      M
4      M
      ..
940    M
941    M
942    M
943    M
944    M
Name: sex, Length: 945, dtype: object

In [23]:
# df = df.drop(columns=['no.', 'eeg.date'], axis=1)
# df = df.fillna(df.mean())
label_encoder = LabelEncoder()

# Encode the categorical target variable y_corr
df['main.disorder']= label_encoder.fit_transform(df['main.disorder'])

# Get the unique categories and their corresponding encoded integers
categories = label_encoder.classes_
encoded_integers = label_encoder.transform(categories)

encoder = OrdinalEncoder()

encoder.fit(df[['sex']])
df['sex']= encoder.transform(df[['sex']])

encoder.fit(df[['specific.disorder']])
df['specific.disorder']= encoder.transform(df[['specific.disorder']])

regex_pattern = '|'.join(['gamma', 'alpha', 'beta'])

# Filter columns based on the concatenated regex pattern
filtered_df = df[df.columns.drop(list(df.filter(regex=regex_pattern)))]

info_df = filtered_df.iloc[:, :6]

regex_pattern = r'(AB|COH)\.[A-Z]\.[theta]*\.[a-z]{1}\.[F].*'

# Select columns based on the regex pattern
selected_columns = filtered_df.filter(regex=regex_pattern)

data = selected_columns
substrings_to_remove = ['FP1', 'F7', 'F8']

# Filter columns and keep only those that do not contain the substrings
filtered_columns = [col for col in data.columns if not any(sub in col for sub in substrings_to_remove)]

# Create a new DataFrame with the filtered columns
filtered_data = data[filtered_columns]

concatenated_df = pd.concat([info_df, filtered_data], axis=1)

data = concatenated_df

data['Suicidal'] = data['main.disorder'].apply(lambda x: 0 if x == 2 else 1)

data = data.drop(columns = ['main.disorder', 'specific.disorder'], axis = 1)


X = data.drop(columns = ['Suicidal'], axis = 1)
y = data['Suicidal']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# rf_classifier = sklearn.ensemble.HistGradientBoostingClassifier(n_estimators=50, random_state=42)
rf_classifier = sklearn.ensemble.HistGradientBoostingClassifier( random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)
df.to_csv('file1.csv')
# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9190140845070423


In [31]:
X.to_csv('data/input_features.csv')
data.to_csv('data/preprocessed_data.csv')

In [25]:
import pickle
pickle.dump(rf_classifier, open('model.pkl', 'wb'))

In [37]:
x=pd.read_csv('C:\Data\DJ\AccioBeer\data\sample2.csv')
output_prediction = rf_classifier.predict(x)

# Print or use the predicted output
print("Predicted Output:", output_prediction)

Predicted Output: [1]
